In [4]:
!nvidia-smi

Sat May 27 06:16:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.48.07    Driver Version: 515.48.07    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:0C:00.0 Off |                  N/A |
| 23%   34C    P8     8W / 250W |    971MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# # mount driver
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/CardiacSeg/eda/Network
#
# #install dependents
# !pip install -q "monai-weekly[nibabel, tqdm, einops]"
# !python -c "import matplotlib" || pip install -q matplotlib
# !pip install -q timm
# !pip install -U -q openmim
# !mim install -U -q mmcv-full
# !pip install ml-collections
# !pip install torchsummaryX
# !pip install ray
# %matplotlib inline

In [53]:
# sync python module
%load_ext autoreload
%autoreload 2

import os
import sys

from torchsummaryX import summary

import torch

# set package path
sys.path.append(r'/nfs/Workspace/CardiacSeg')

import argparse
from networks.network import network
from networks.networkx.blocks.cbam import CBAM
from networks.networkx.blocks.ham import HAM
from networks.ssl_head import SSLHead

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
# import torchvision.models as models
# import torch
# from ptflops import get_model_complexity_info


# macs, params = get_model_complexity_info(
#     net, 
#     (1, 128, 128, 128), 
#     as_strings=True,
#     print_per_layer_stat=False, 
#     verbose=False
# )
# print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
# print('{:<30}  {:<8}'.format('Number of parameters: ', params))

In [57]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [128, 128, 128]
args.patch_size = 4
args.drop_rate = 0.1
args.kernel_size = 7
args.exp_rate = 4
args.norm_name = 'layer'
args.depths = [3, 3, 9, 3]
args.feature_size = 48         
args.use_init_weights = False
args.deep_sup=True
args.is_conv_stem=False
args.first_feature_size_half=False
args.skip_encoder_name=None #'convmod' #'eca'



net = None
del net
net = network('cotr', args) # unetc2f

# inp = None
# del inp
# inp = torch.rand((1, 1, args.roi_x, args.roi_y, args.roi_z)).to(args.device)
# summary(net, inp)

log_dir = '/nfs/Workspace/CardiacSeg/exps/exps/cotr/chgh/tune_results/t_4/main_0ef8c_00000_0_exp=exp_t_4_2023-04-02_10-06-38'
model_pth = os.path.join(log_dir, 'models', 'final_model.pth')
ckp = torch.load(model_pth)
print('final early stop count:', ckp['early_stop_count'])
print('final epoch:', ckp['epoch'])
print('best val dice:', ckp['best_acc'])
net.load_state_dict(ckp["state_dict"])

model: cotr


nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


  + Number of Backbone Params: 20.50(e6)
  + Number of Transformer Params: 9.32(e6)
final early stop count: 20
final epoch: 700
best val dice: 0.8893863558769226


<All keys matched successfully>

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [128, 128, 128]
args.patch_size = 4
args.drop_rate = 0.1
args.kernel_size = 7
args.exp_rate = 4
args.norm_name = 'layer'
args.depths = [4, 4, 4, 4]
args.feature_size = 48         
args.use_init_weights = False
args.deep_sup=True
args.is_conv_stem=True
args.skip_encoder_name='cbam'
args.deep_sup=True
net = None
del net
net = network('unetcnx_a1', args) # unetc2f
inp = None
del inp
inp = torch.rand((1, 1, 128, 128, 128)).to(args.device)
summary(net, inp)

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [128, 128, 128]
net = None
del net
net = network('unetcnx_x2', args)
inp = None
del inp
inp = torch.rand((1, 1, 128, 128, 128)).to(args.device)
summary(net, inp)

## DenseVoxNet

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
net = network('dense_vox_net', args)
inp = torch.rand((1, 1, 96, 96, 96)).to(args.device)
summary(net, inp)

## EffcientSegNet

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
net = network('efficient_segnet_x0_2', args)
summary(net, (1, 96, 96, 96))

## UNETCNX

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
net = network('unetcnx_x0', args)
model = SSLHead(args.in_channels, net).to(args.device)
summary(model, (1, 96, 96, 96))

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
model = network('unetcnx_x1', args)
summary(model, (1, 96, 96, 96))